In [1]:
%load_ext autoreload
%autoreload 2
import jieba
import transformers as ts
import pandas as pd
import numpy as np
import torch
import json
from sklearn.model_selection import train_test_split
from ark_nlp.model.ner.global_pointer_bert import Dataset,Tokenizer,GlobalPointerBertConfig,GlobalPointerBert,get_default_model_optimizer,Task,Predictor
from torch.utils.data import DataLoader,Dataset

In [2]:
df_train=pd.read_json('./datasets/train.json')
df_train.head(3)

,text,ner_list,type,doc_id
0,血管内皮生长因子表达水平与宫颈癌患者预后的关系,"[{'mention': '血管内皮生长因子表达水平与宫颈癌患者预后的关系', 'label...",Title,2500
1,目的探讨地尔硫卓在治疗扩张型心肌病中的临床疗效。方法将2014年9月至2017年12月在红河...,"[{'mention': '探讨地尔硫卓在治疗扩张型心肌病中的临床疗效', 'label':...",Abstract,2501
2,地尔硫卓在治疗扩张型心肌病中的临床疗效观察,"[{'mention': '地尔硫卓在治疗扩张型心肌病中的临床疗效观察', 'label':...",Title,2501


In [27]:
title_data=[]
context_data=[]
lens=len(df_train)
for i in range(0,lens):
    if df_train.loc[i,'type'] == 'Title':
        text=df_train.loc[i,'text'].replace(' ','')
        label=[]
        context_data.append({
                'text':text,
                'label':df_train.loc[i,'ner_list'][0]['label'],
            })
        for ner in df_train.loc[i,'ner_list']:
            subtext=ner['mention'].replace(' ','')
            sidx=text.find(subtext)
            if sidx == -1:
                continue
            eidx=sidx+len(subtext)
            label.append({
                'start_idx':sidx,
                'end_idx':eidx,
                'type':ner['label'],
                'entity':subtext,
            })
        title_data.append({
            'text':text,
            'label':label
        })
    else:
        for ner in df_train.loc[i,'ner_list']:
            context_data.append({
                'text':ner['mention'],
                'label':ner['label'],
            })
title_data=json.dumps(title_data,ensure_ascii = False)
context_data=json.dumps(context_data,ensure_ascii = False)
# with open('./datasets/title_train.json','w',encoding='utf-8')as f:
#     f.write(title_data)
#     f.close()
with open('./datasets/context_title_train.json','w',encoding='utf-8')as f:
    f.write(context_data)
    f.close()

In [2]:
label_list=['C-对比选项',
 'C-研究方法',
 'C-研究目的',
 'I-其他干预',
 'I-干预',
 'I-教育/行为干预',
 'I-药物干预',
 'I-非药物干预',
 'O-定性结论',
 'O-定量结论',
 'O-结论',
 'O-结论主语',
 'P-人群/患者类型',
 'P-条件',
 'P-研究对象',
 'P-评估项',
 'S-因素(病因/风险)分析',
 'S-指南标准建议',
 'S-治疗',
 'S-病因学',
 'S-统计分析',
 'S-诊断',
 'S-预后']

In [4]:
title_data=pd.read_json('./datasets/title_train.json')
title_data=title_data.iloc[0:100,:]
title_data['text'].str.len().max()

45

In [5]:
train_set,val_set=train_test_split(title_data,test_size=0.2,random_state=1234)
train_set=train_set.reset_index(drop=True)
val_set=val_set.reset_index(drop=True)
train_set['label']=train_set['label'].apply(lambda x: str(x))
val_set['label']=val_set['label'].apply(lambda x: str(x))
len(train_set),len(val_set)

(80, 20)

In [8]:
trainds=Dataset(train_set,categories=label_list)
valds=Dataset(val_set,categories=label_list)

In [7]:
tokenizer=ts.BertTokenizer.from_pretrained('../../model/macbert-base-chinese-medical-collation/')
ttokenier=Tokenizer(vocab=tokenizer,max_seq_len=100)

In [10]:
trainds.convert_to_ids(ttokenier)
valds.convert_to_ids(ttokenier)

In [14]:
trainds.dataset[0]

{'input_ids': array([ 101, 7770, 7574, 4510, 1143, 3780, 4545, 2714, 2595, 2151, 7568,
        4142, 9653,  891,  102,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0], dtype=int64),
 'attention_mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [24]:
from model import GlobalPointer,GlobalPointerCrossEntropy,GlobalPointerNERPredictor
args={
    'bert_dir':'../../model/macbert-base-chinese-medical-collation/'
}
bertModel=GlobalPointer(args,len(label_list),64)

Some weights of BertModel were not initialized from the model checkpoint at ../../model/macbert-base-chinese-medical-collation/ and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
train_dl=DataLoader(trainds.dataset,batch_size=16)
loss_fn=GlobalPointerCrossEntropy()
opt=torch.optim.Adam(bertModel.parameters(),lr=5e-5)
for e in range(0,10):
    train_loss=0
    for step,bd in enumerate(train_dl):
        ppp=bertModel(bd['input_ids'],bd['attention_mask'],bd['token_type_ids'])
        loss=loss_fn(ppp,bd['label_ids'].to_dense())
        train_loss+=loss.item()
        opt.zero_grad()
        loss.backward()
        opt.step()
        
    print('epoch:',e,'  loss:',train_loss/step)     

epoch: 0   loss: 6.85793399810791
epoch: 1   loss: 6.169060945510864
epoch: 2   loss: 5.330308437347412
epoch: 3   loss: 4.254041850566864
epoch: 4   loss: 3.0130489468574524
epoch: 5   loss: 1.8222541809082031
epoch: 6   loss: 1.113606721162796
epoch: 7   loss: 0.8432368859648705
epoch: 8   loss: 0.6961652711033821
epoch: 9   loss: 0.5927189439535141


In [26]:
test_text='常染色体显性多囊肾病的临床问题及其肾脏替代治疗的选择'
mpred=GlobalPointerNERPredictor(bertModel,ttokenier,trainds.cat2id,tokenizer)
result=mpred.predict_one_sample(test_text)
result

[]

In [13]:
import torch.nn as nn
class clf(nn.Module):
    """docstring for clf."""
    def __init__(self, bert_path,cat_num):
        super(clf, self).__init__()
        bcfg=ts.BertConfig.from_pretrained(bert_path)
        self.bert=ts.BertModel.from_pretrained(bert_path,config=bcfg)
        self.mlp=nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(768,512,),
            nn.GELU(),
            nn.Linear(512,cat_num),
            nn.GELU(),
            nn.Softmax(dim=1)
        )
        
    def forward(self,input_ids,attention_mask,token_type_ids):
        last_hidden_state=self.bert(input_ids,attention_mask,token_type_ids)
        logits=last_hidden_state[0][:,0,:]
        logits=self.mlp(logits)
        return logits
    

class medData(Dataset):
    def __init__(self,texts,labels,tokenizer,maxlen,label_list) -> None:
        super(medData,self).__init__()
        self.text=[tokenizer(t,
                            padding='max_length',
                            max_length = maxlen) for t in texts]
        self.labels=[label_list.index(l) for l in labels]
        
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, index):
        input_ids=torch.LongTensor(self.text[index]['input_ids'])
        attention_mask=torch.LongTensor(self.text[index]['attention_mask'])
        token_type_ids=torch.LongTensor(self.text[index]['token_type_ids'])
        label=torch.LongTensor([self.labels[index]])
        return {'input_ids':input_ids,'attention_mask':attention_mask,'token_type_ids':token_type_ids},label

In [3]:
ctrain_df=pd.read_json('./datasets/context_title_train.json')
ctrain_df=ctrain_df.iloc[0:200,:]
ctrain_df.head(3)

,text,label
0,血管内皮生长因子表达水平与宫颈癌患者预后的关系,S-诊断
1,探讨地尔硫卓在治疗扩张型心肌病中的临床疗效,C-研究目的
2,将2014年9月至2017年12月在红河州第二人民医院内二科住院治疗的40例扩张型心肌病患者...,C-研究方法


In [8]:
ctrain_dataset=medData(ctrain_df['text'].__array__(),ctrain_df['label'].__array__(),tokenizer,128,label_list)
ctrain_dl=DataLoader(ctrain_dataset,batch_size=16)

In [12]:
tokenizer.decode(ctrain_dataset.text[0]['input_ids'])

'[CLS] 血 管 内 皮 生 长 因 子 表 达 水 平 与 宫 颈 癌 患 者 预 后 的 关 系 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [14]:
classfier=clf('../../model/macbert-base-chinese-medical-collation/',len(label_list))

Some weights of BertModel were not initialized from the model checkpoint at ../../model/macbert-base-chinese-medical-collation/ and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for step,(bd,l) in enumerate(ctrain_dl):
    logits=classfier(bd['input_ids'],bd['attention_mask'],bd['token_type_ids'])
    pred=torch.max(logits,dim=-1)
    #(b,n,c)——(b,1)